In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy import optimize
from scripts.estimation import *
from scripts.utils import *
from tqdm import tqdm
import time

## Comparsion of the EM and MM methods

The following snippet reproduces table 1 in the paper. It compares the performance of the EM algorithm with the MM algorithm. 

In [2]:
lambd = 10
grid_size = 5
beta0 = 1
beta1_true = -1
T = 100
loc_bound = 5
EM_time = 0
MM_time = 0
num_rep = 10
EM_iter = 0
MM_iter = 0
EM_lkd = 0
MM_lkd = 0
EM_wd = 0
MM_wd = 0

for rand_seed in tqdm(range(num_rep)):
    num_position = 10 # number of arrival locations
    bike_num = 40 # total number of bikes

    true_pos_ind,bike_num,num_records,book_bike,book_index, \
    dist,bike_loc,all_period,num_booked, \
    cand_loc,true_loc,position_weight = gen_sync_in_grid(rand_seed,num_position,bike_num,lambd,
                                                        grid_size,beta0=beta0,beta1_true=beta1_true,T=T,loc_bound=loc_bound)
    # Initialization of parameters
    num_position_true = num_position
    num_position = grid_size**2

    w = np.random.uniform(0,1,num_position)
    w = w/np.sum(w)
    w = w.reshape(1,-1)
    diff_w = np.inf
    beta1 = np.repeat(-1,num_position).reshape(-1,1)
    dist_old = caldist(cand_loc[-1],bike_loc,bike_num)
    iter = 0
    threshold = 1e-3

    p_old = np.zeros((num_position,bike_num+1,num_records))
    p_old[:,0,:] = 1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))
    p_old[:,1:,:] = np.exp(beta0+np.reshape(beta1,(-1,1,1))*np.transpose(dist_old,(0,2,1)))/ \
        np.reshape((1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2)),(num_position,1,-1))
    p1 = np.sum(p_old[:,0,:]*all_period,axis=1)
    p2 = (1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2)))
    # EM iteration
    EM_t_start = time.time()
    while diff_w > threshold:
        iter = iter+1
        p_olds = np.zeros((num_position,num_booked))
        w_old = w[-1,:]
        p0 = p_old[:,book_bike+1,book_index]*np.reshape(w_old,(-1,1))
        p_olds = p0/np.reshape(np.sum(p0,axis=0),(1,-1))
        p_oldsj0 = w_old*p1/np.sum(w_old*p1)
        s_old = np.sum((1-np.sum(np.expand_dims(w_old,1)*p2,axis=0))*all_period)
        N_oldy = num_booked*(T-s_old)/s_old
        c = np.sum(p_olds,axis=1)+N_oldy*p_oldsj0
        w_star = np.reshape(c/np.sum(c),(1,-1))
        w = np.concatenate((w,w_star),axis=0)
        diff_w = np.sum(np.abs(w[-1,:]-w[-2,:]))
    
    EM_t_end = time.time()
    EM_time += EM_t_end - EM_t_start

    pre_w = w[-2]
    true_w = np.zeros(grid_size**2)
    true_w[true_pos_ind] = position_weight
    lkd = findlkd_no_constraint(num_position,dist_old,
                                    beta0,np.repeat(beta1_true,num_position).reshape(-1,1),pre_w,
                                    bike_num,num_records,book_bike,book_index,num_booked,all_period)
    was_dist = find_wasserstein(cand_loc[0,:,:],true_loc,pre_w,position_weight)[0]

    EM_iter += iter
    EM_lkd += lkd
    EM_wd += was_dist

    # Initialization of parameters
    num_position = grid_size**2
    w = np.random.uniform(0,1,num_position)
    w = w/np.sum(w)
    w = w.reshape(1,-1)
    beta1 = np.repeat(-1,num_position).reshape(-1,1)
    loc = cand_loc

    dist1 = caldist(loc,bike_loc,bike_num)
    choice_prob = np.zeros((num_position,bike_num+1,num_records))
    choice_prob[:,0,:] = 1/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2))
    choice_prob[:,1:,:] = np.exp(beta0+beta1.reshape(-1,1,1)*np.transpose(dist1,(0,2,1)))/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2)).reshape(num_position,1,-1)
    diff_w = np.inf
    threshold = 1e-4
    threshold1 = 5e-3
    iter = 0
    grad_old = 0
    diff_grad = np.inf
    w_prime = w

    # Define the function that is optimized using golden-section search method
    def gold_search_func(alpha):
        f = g(w_prime+alpha*(ej-w_prime),choice_prob,book_bike,book_index)-np.sum(grad_h*(w_prime+alpha*(ej-w_prime)))
        return -f

    MM_t_start = time.time()

    # Frank-Wolfe algorithm
    while(diff_w>threshold):
        iter += 1
        gw_prime = g(w_prime,choice_prob,book_bike,book_index)
        grad_h = -num_booked *np.sum(choice_prob[:,0,:]*all_period,axis=1)/np.sum((1-np.sum(w.reshape(-1,1)*choice_prob[:,0,:],axis=0))*all_period)
        diff_grad = np.inf
        while (diff_grad>threshold1):
            grad_g = find_grad_g(w_prime,choice_prob,book_bike,book_index)
            j = np.argmax(grad_g-grad_h)
            ej = np.zeros(num_position)
            ej[j] = ej[j]+1
            alpha = optimize.golden(gold_search_func, brack=(0, 1))
            w_prime = w_prime + alpha*(ej-w_prime)
            grad_new = np.abs(gw_prime-np.sum(grad_h*w_prime))
            diff_grad = np.abs(grad_new-grad_old)
            grad_old = grad_new
        diff_w = np.sum(np.abs(w_prime-w))
        w = w_prime

    MM_t_end = time.time()
    MM_time += MM_t_end - MM_t_start

    pre_w = w[-1]
    true_w = np.zeros(grid_size**2)
    true_w[true_pos_ind] = position_weight
    l1norm = np.sum(np.abs(pre_w-true_w))
    lkd = findlkd_no_constraint(num_position,dist_old,
                                    beta0,np.repeat(beta1_true,num_position).reshape(-1,1),pre_w,
                                    bike_num,num_records,book_bike,book_index,num_booked,all_period)
    was_dist = find_wasserstein(cand_loc[0,:,:],true_loc,pre_w,position_weight)[0]
    MM_iter += iter
    MM_lkd += lkd
    MM_wd += was_dist

    print(EM_time,MM_time)

print(EM_time/num_rep, MM_time/num_rep)
print(EM_iter/num_rep, MM_iter/num_rep)
print(EM_lkd/num_rep, MM_lkd/num_rep)
 

  0%|          | 0/10 [00:00<?, ?it/s]

/home/azureuser/bike_sharing_new/bike_sharing_upload/bike_sharing/scripts/estimation.py:353: RuntimeWarning: invalid value encountered in log
  g = np.sum(np.log(np.sum(w.reshape(-1,1)*(choice_prob[:,book_bike+1,book_index]),axis=0)))
 10%|█         | 1/10 [00:14<02:14, 14.96s/it]

0.017105579376220703 13.873564720153809


 20%|██        | 2/10 [00:40<02:51, 21.46s/it]

0.04201912879943848 39.39405012130737


...

0.16292977333068848 173.8023760318756


 90%|█████████ | 9/10 [03:20<00:21, 21.48s/it]

0.18083620071411133 195.38119912147522


100%|██████████| 10/10 [03:37<00:00, 21.80s/it]

0.20245623588562012 212.95745086669922
0.020245623588562012 21.29574508666992
120.2 274.8
-4533.5831768438175 -4533.467803038004


In [3]:
print('Avg. time for EM/MM algorithm:',EM_time/num_rep, MM_time/num_rep)
print('Avg. num. of iterations for EM/MM algorithm:',EM_iter/num_rep, MM_iter/num_rep)
print('Avg. likelihood for EM/MM algorithm:',EM_lkd/num_rep, MM_lkd/num_rep)
print('Avg. Wasserstein distance for EM/MM algorithm:',EM_wd/num_rep, MM_wd/num_rep)

Avg. time for EM/MM algorithm: 0.020245623588562012 21.29574508666992
Avg. num. of iterations for EM/MM algorithm: 120.2 274.8
Avg. likelihood for EM/MM algorithm: -4533.5831768438175 -4533.467803038004
Avg. Wasserstein distance for EM/MM algorithm: 3.4529780622510997 3.4547998173448873
